## Setups
Mount (and unmount before) a folder on ADLS where the data was put by the ADF-Pipeline

See https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal  how the Azure Active Directory App ID is created with its credentials. Please note: DON'T FORGET TO COPY THE KEY BEFORE YOU SAVE IT, YOU WON'T BE ABLE TO ACCESS IT AFTERWARDS!!!

In [2]:
#dbutils.fs.unmount("/mnt/airdelays")

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3807129141078225> in <module> () 
 ----> 1 dbutils . fs . unmount ( "/mnt/airdelays" ) 

 /local_disk0/tmp/1553635215488-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 302 exc . __context__ = None 
 303 exc . __cause__ = None 
 --> 304 raise exc 
 305 return f_with_exception_handling 
 306 

 ExecutionError : An error occurred while calling o210.unmount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: Directory not mounted: /mnt/airdelays; nested exception is: 
	java.lang.IllegalArgumentException: Directory not mounted: /mnt/airdelays
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:54)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:541)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:552)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: Directory not mounted: /mnt/airdelays
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.deleteMount(MetadataManager.scala:154)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:88)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:83)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:82)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:82)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:272)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:252)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:42)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:38)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$2(UsageLogging.scala:359)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:13)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:268)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:264)
	at com.databricks.rpc.ServerBackend.withAttributionTags(ServerBackend.scala:13)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:345)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.sca

In [3]:
configs = {"dfs.adls.oauth2.access.token.provider.type": "ClientCredential",
           "dfs.adls.oauth2.client.id": "[YOURAPPID]",  #this is the appid
           "dfs.adls.oauth2.credential": "[YOURKEY]", #there goes the key created
           "dfs.adls.oauth2.refresh.url": "https://login.microsoftonline.com/[YOURDIRECTORY]/oauth2/token"} #this is the Tenant- or Directory ID

dbutils.fs.mount(
  source = "adl://[YOURDATALAKESTORE].azuredatalakestore.net/[YOURFOLDER]",
  mount_point = "/mnt/airdelays",
  extra_configs = configs)

Out[ 13 ]: True

## But let's start playing around. 
We'll create a dataframe from the content of the "airdelays" folder. 

Assumption is, the files look the same. (if not, there are methods for this ;) )

In [5]:
csvFolder = "/mnt/airdelays"
df = (spark.read                        # The DataFrameReader
   .option("header", "true")            # Use first line of all files as header
   .option("inferSchema", "true")       # Automatically infer data types
   .csv(csvFolder)        
                                        # Creates a DataFrame from CSV after reading in the file
)

Cache the dataframe to speed up the following steps

In [7]:
df.cache()

Out[ 15 ]: DataFrame[YEAR: int, MONTH: int, DAY_OF_MONTH: int, DAY_OF_WEEK: int, FL_DATE: timestamp, UNIQUE_CARRIER: string, TAIL_NUM: string, FL_NUM: int, ORIGIN_AIRPORT_ID: int, ORIGIN: string, ORIGIN_STATE_ABR: string, DEST_AIRPORT_ID: int, DEST: string, DEST_STATE_ABR: string, CRS_DEP_TIME: int, DEP_TIME: int, DEP_DELAY: double, DEP_DELAY_NEW: double, DEP_DEL15: double, DEP_DELAY_GROUP: int, TAXI_OUT: double, WHEELS_OFF: string, WHEELS_ON: string, TAXI_IN: double, CRS_ARR_TIME: int, ARR_TIME: int, ARR_DELAY: double, ARR_DELAY_NEW: double, ARR_DEL15: double, ARR_DELAY_GROUP: int, CANCELLED: double, CANCELLATION_CODE: string, DIVERTED: double, CRS_ELAPSED_TIME: double, ACTUAL_ELAPSED_TIME: double, AIR_TIME: double, FLIGHTS: double, DISTANCE: double, DISTANCE_GROUP: int, CARRIER_DELAY: double, WEATHER_DELAY: double, NAS_DELAY: double, SECURITY_DELAY: double, LATE_AIRCRAFT_DELAY: double, _c44: string]

Need the distinct values of a dataframe... 

Here (https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html) you can find some documentation about PySpark.

In [9]:
display(df.select("ORIGIN").distinct())

ORIGIN
BGM
DLG
PSE
INL
MSY
GEG
DRT
BUR
SNA
GRB


## Next steps: 

- drop unnecessary columns from the dataframe
- eliminate or replace Nulls from the dataframe?
- create new column with a new value
- visually check some values
- create a job out of the workbook
- write out the results as a parquet file
- maybe add some streaming data?

In [11]:
delays = df.select( \
          "YEAR", \
          "MONTH", \
          "DAY_OF_MONTH", \
          "DAY_OF_WEEK", \
          "ORIGIN", \
          "UNIQUE_CARRIER", \
          "ARR_DELAY", \
          "ARR_DEL15", \
          "ORIGIN_STATE_ABR", \
          "DEST_STATE_ABR", \
          "DEP_DELAY"\
         )

In [12]:
delays.cache()

Out[ 20 ]: DataFrame[YEAR: int, MONTH: int, DAY_OF_MONTH: int, DAY_OF_WEEK: int, ORIGIN: string, UNIQUE_CARRIER: string, ARR_DELAY: double, ARR_DEL15: double, ORIGIN_STATE_ABR: string, DEST_STATE_ABR: string, DEP_DELAY: double]

In [13]:
display(delays)

In [14]:
from pyspark.sql.functions import isnan, when, count, col 
delays.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in delays.columns]).show()

+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|ORIGIN|UNIQUE_CARRIER|ARR_DELAY|ARR_DEL15|ORIGIN_STATE_ABR|DEST_STATE_ABR|DEP_DELAY|
+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
 0| 0| 0| 0| 0| 0| 3042918| 3042917| 58508| 57794| 2683555|
+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+

## NULL Handling
Do we create additional bias into our data?

In [16]:
# Do we replace NULL values? (in this case with Zero = 0)
delaysfilna = delays.fillna(0)

# Or do we eliminate them?
delaysnona = delays.dropna()
delaysnona.cache()

In [17]:
delaysnona.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in delays.columns]).show()

+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|ORIGIN|UNIQUE_CARRIER|ARR_DELAY|ARR_DEL15|ORIGIN_STATE_ABR|DEST_STATE_ABR|DEP_DELAY|
+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+

In [18]:
display(delaysnona)

also do some calcs...

In [20]:
delaysnona_newc = delaysnona.withColumn("DELTA", delaysnona['ARR_DELAY'] - delaysnona['DEP_DELAY']*-1)
delaysnona_newc.cache()

display(delaysnona_newc)


YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,UNIQUE_CARRIER,ARR_DELAY,ARR_DEL15,ORIGIN_STATE_ABR,DEST_STATE_ABR,DEP_DELAY,DELTA
2004,7,1,4,JFK,AA,8.0,0.0,NY,CA,0.0,8.0
2004,7,2,5,JFK,AA,15.0,1.0,NY,CA,-2.0,13.0
2004,7,3,6,JFK,AA,2.0,0.0,NY,CA,-6.0,-4.0
2004,7,4,7,JFK,AA,-2.0,0.0,NY,CA,-5.0,-7.0
2004,7,5,1,JFK,AA,1.0,0.0,NY,CA,-7.0,-6.0
2004,7,6,2,JFK,AA,6.0,0.0,NY,CA,3.0,9.0
2004,7,7,3,JFK,AA,-2.0,0.0,NY,CA,-8.0,-10.0
2004,7,8,4,JFK,AA,31.0,1.0,NY,CA,-2.0,29.0
2004,7,9,5,JFK,AA,20.0,1.0,NY,CA,-7.0,13.0
2004,7,10,6,JFK,AA,40.0,1.0,NY,CA,41.0,81.0


## Create temporary View ==> for the later use with SQL

In [22]:
delaysnona_newc.createOrReplaceTempView("airdelays_table")

Now we just switch to SQL 
(could also be R or Scala btw)

In [24]:
%sql

select *
from airdelays_table


YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,UNIQUE_CARRIER,ARR_DELAY,ARR_DEL15,ORIGIN_STATE_ABR,DEST_STATE_ABR,DEP_DELAY,DELTA
2004,7,1,4,JFK,AA,8.0,0.0,NY,CA,0.0,8.0
2004,7,2,5,JFK,AA,15.0,1.0,NY,CA,-2.0,13.0
2004,7,3,6,JFK,AA,2.0,0.0,NY,CA,-6.0,-4.0
2004,7,4,7,JFK,AA,-2.0,0.0,NY,CA,-5.0,-7.0
2004,7,5,1,JFK,AA,1.0,0.0,NY,CA,-7.0,-6.0
2004,7,6,2,JFK,AA,6.0,0.0,NY,CA,3.0,9.0
2004,7,7,3,JFK,AA,-2.0,0.0,NY,CA,-8.0,-10.0
2004,7,8,4,JFK,AA,31.0,1.0,NY,CA,-2.0,29.0
2004,7,9,5,JFK,AA,20.0,1.0,NY,CA,-7.0,13.0
2004,7,10,6,JFK,AA,40.0,1.0,NY,CA,41.0,81.0


In [25]:
%sql

select
  ORIGIN
  , avg(ARR_DELAY)
from airdelays_table
group by ORIGIN

ORIGIN,avg(ARR_DELAY)
BGM,3.338702543309989
PSE,-0.7377800407331976
INL,5.622596153846154
DLG,10.86890182694238
MSY,5.167156959000795
GEG,4.104301394051742
DRT,-6.914364640883978
SNA,4.164027677504417
BUR,4.34880618953682
GRB,4.205632628307179


In [26]:
%sql

select 
  UNIQUE_CARRIER
  , (ARR_DELAY - DEP_DELAY) as DELTA
from airdelays_table

UNIQUE_CARRIER,DELTA
AA,8.0
AA,17.0
AA,8.0
AA,3.0
AA,8.0
AA,3.0
AA,6.0
AA,33.0
AA,27.0
AA,-1.0


## But enough with that, let's output the results. First switch back to a PySpark Dataframe

In [28]:
dfairdelays_new = spark.table("airdelays_table")

In [29]:
dfairdelays_new.cache()

Out[ 26 ]: DataFrame[YEAR: int, MONTH: int, DAY_OF_MONTH: int, DAY_OF_WEEK: int, ORIGIN: string, UNIQUE_CARRIER: string, ARR_DELAY: double, ARR_DEL15: double, ORIGIN_STATE_ABR: string, DEST_STATE_ABR: string, DEP_DELAY: double, DELTA: double]

## and then remove the file if there and write it to the Data lake Store in Parquet-Format

In [31]:
#dbutils.fs.rm("/mnt/airdelays/airdelays_table.parquet", True)
dfairdelays_new.write.parquet("/mnt/airdelays/airdelays_table.parquet")

In [32]:
dfairdelays_new

Out[ 28 ]: DataFrame[YEAR: int, MONTH: int, DAY_OF_MONTH: int, DAY_OF_WEEK: int, ORIGIN: string, UNIQUE_CARRIER: string, ARR_DELAY: double, ARR_DEL15: double, ORIGIN_STATE_ABR: string, DEST_STATE_ABR: string, DEP_DELAY: double, DELTA: double]

## But wait, how many rows did we just write to our Data Lake and how long did it take?

In [34]:
dfairdelays_new.count()

Out[ 29 ]: 145479068

## OK, then lets play a little with Python
First: very boring show() the content of the dataframe

In [36]:
delays.show()

+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|ORIGIN|UNIQUE_CARRIER|ARR_DELAY|ARR_DEL15|ORIGIN_STATE_ABR|DEST_STATE_ABR|DEP_DELAY|
+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
2004| 7| 1| 4| JFK| AA| 8.0| 0.0| NY| CA| 0.0|
2004| 7| 2| 5| JFK| AA| 15.0| 1.0| NY| CA| -2.0|
2004| 7| 3| 6| JFK| AA| 2.0| 0.0| NY| CA| -6.0|
2004| 7| 4| 7| JFK| AA| -2.0| 0.0| NY| CA| -5.0|
2004| 7| 5| 1| JFK| AA| 1.0| 0.0| NY| CA| -7.0|
2004| 7| 6| 2| JFK| AA| 6.0| 0.0| NY| CA| 3.0|
2004| 7| 7| 3| JFK| AA| -2.0| 0.0| NY| CA| -8.0|
2004| 7| 8| 4| JFK| AA| 31.0| 1.0| NY| CA| -2.0|
2004| 7| 9| 5| JFK| AA| 20.0| 1.0| NY| CA| -7.0|
2004| 7| 10| 6| JFK| AA| 40.0| 1.0| NY| CA| 41.0|
2004| 7| 11| 7| JFK| AA| -17.0| 0.0| NY| CA| -4.0|
2004| 7| 12| 1| JFK| AA| 20.0| 1.0| NY| CA| 0.0|
2004| 7| 13| 2| JFK| AA| -17.0| 0.0| NY| CA| -4.0|
2004| 7| 14| 3| JFK| AA| 48.0| 1.0| NY| CA| -2.0|
2004| 7| 15| 4| JFK| AA| 27.0| 1.0| NY| CA| 55.0|
2004| 7| 16| 5| JFK| AA| -4.0| 0.0| NY| CA| -3.0|
2004| 7| 17| 6| JFK| AA| 13.0| 0.0| NY| CA| -7.0|
2004| 7| 18| 7| JFK| AA| 1.0| 0.0| NY| CA| -4.0|
2004| 7| 19| 1| JFK| AA| 1.0| 0.0| NY| CA| 8.0|
2004| 7| 20| 2| JFK| AA| -10.0| 0.0| NY| CA| -7.0|
+----+-----+------------+-----------+------+--------------+---------+---------+----------------+--------------+---------+
only showing top 20 rows

Then: let's have a more sophisticated view on the things: display([YOURDATAFRAME])

In [38]:
display(delays)

Count() the rows in the df

In [40]:
delays.count()

Cache the df into the memory of the cluster to speed things up a little

In [42]:
delays.cache()

Now do some math (the Data Scientists will know, where this is heading...)
Note the small buttons below the chart? ==> Here you can toggle table or chart-view after you have displayed the dataframe

In [44]:
display(delays.groupby('DAY_OF_WEEK').avg('ARR_DELAY'))

DAY_OF_WEEK,avg(ARR_DELAY)
1,6.365682122137005
6,3.7567615734038102
3,6.538510925600507
5,8.977518779083464
4,8.40099970359445
7,6.062001282397349
2,5.472585296998161


In [45]:
display(delays.groupby('UNIQUE_CARRIER').avg('ARR_DELAY'))

But can we create a python function and use it?

In [47]:
def patsum( arg1, arg2 ):
  # Add both the parameters and return them."
  total = arg1 + arg2
  print("Inside the function : ", total)
  return total;

In [48]:
display(delays.withColumn('Summenfeld', patsum(col('ARR_DELAY'), col('ARR_DEL15'))))

YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,UNIQUE_CARRIER,ARR_DELAY,ARR_DEL15,ORIGIN_STATE_ABR,DEST_STATE_ABR,DEP_DELAY,Summenfeld
2004,7,1,4,JFK,AA,8.0,0.0,NY,CA,0.0,8.0
2004,7,2,5,JFK,AA,15.0,1.0,NY,CA,-2.0,16.0
2004,7,3,6,JFK,AA,2.0,0.0,NY,CA,-6.0,2.0
2004,7,4,7,JFK,AA,-2.0,0.0,NY,CA,-5.0,-2.0
2004,7,5,1,JFK,AA,1.0,0.0,NY,CA,-7.0,1.0
2004,7,6,2,JFK,AA,6.0,0.0,NY,CA,3.0,6.0
2004,7,7,3,JFK,AA,-2.0,0.0,NY,CA,-8.0,-2.0
2004,7,8,4,JFK,AA,31.0,1.0,NY,CA,-2.0,32.0
2004,7,9,5,JFK,AA,20.0,1.0,NY,CA,-7.0,21.0
2004,7,10,6,JFK,AA,40.0,1.0,NY,CA,41.0,41.0


OK, let's go to where the Data Scientists feel more comfortable...

describe() will show the most important stats for the dataframe given

In [50]:
delays.describe('ARR_DELAY').show()

+-------+------------------+
summary| ARR_DELAY|
+-------+------------------+
 count| 145576737|
 mean| 6.566599511019401|
 stddev|31.556410693714593|
 min| -1437.0|
 max| 2598.0|
+-------+------------------+

And we can find a correlation between two values. This doesn't need to be important and can be a coincidence.

In [52]:
delays.corr('ARR_DELAY', 'DEP_DELAY')

Out[ 35 ]: 0.8760176256940212

From here on the Data Scientists might take over ==> more to come in the Advanced Analytics and Machine Learning Lab... ;)